In [1]:
import pandas as pd
import json
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
colnames=['tweets', 'created_at', 'id', 'rt', 'lang', 'urls', 'Event']
df_relevant = pd.read_csv('clean_relevant_tweets.csv', header='infer')
df_clusters = pd.read_csv('dbscan_TFIDF_05.csv',header='infer',engine='python')


colnames=['tweets', 'created_at', 'id', 'rt', 'lang', 'urls'] 
df_irrelevant = pd.read_csv('output_irrelevant_nocord.csv', names=colnames, header=None)
frames = [df_relevant,df_irrelevant]
df = pd.concat(frames)
df = df.loc[df["lang"]=='en']


import timestamp_graphs as tg
df = df.reset_index(drop=True)
df = tg.createTimestamps(df)

list_date_minutes = tg.getList(df,'Datetime')
print(len(list_date_minutes))

39949


In [3]:
df.head()

,tweets,created_at,id,rt,lang,urls,Event,date,time,Datetime,DateHour,Date_Ten_Minutes
0,News: Chinese author Mo Yan wins Nobel http://...,Thu Oct 11 11:08:32 +0000 2012,256350567369175040,False,en,http://bbc.in/RbTlMC,0.0,2012-10-11,11:08:32,2012-10-11 11:08:00,2012-10-11 11:00:00,2012-10-11 11:00:00
1,CCTV invite hints at Nobel literature prize fo...,Thu Oct 11 10:51:28 +0000 2012,256346272506712064,False,en,http://bit.ly/RhMRvS,0.0,2012-10-11,10:51:28,2012-10-11 10:51:00,2012-10-11 10:00:00,2012-10-11 10:50:00
2,CCTV invite hints at Nobel literature prize fo...,Thu Oct 11 10:51:28 +0000 2012,256346272506712064,False,en,http://bit.ly/RhMRvS,0.0,2012-10-11,10:51:28,2012-10-11 10:51:00,2012-10-11 10:00:00,2012-10-11 10:50:00
3,Mo Yan wins 2012 Nobel Prize for Literature. G...,Thu Oct 11 11:11:22 +0000 2012,256351280652488704,False,en,http://bit.ly/SNAqLI,0.0,2012-10-11,11:11:22,2012-10-11 11:11:00,2012-10-11 11:00:00,2012-10-11 11:10:00
4,Mo Yan wins 2012 Nobel Prize for Literature. G...,Thu Oct 11 11:11:22 +0000 2012,256351280652488704,False,en,http://bit.ly/SNAqLI,0.0,2012-10-11,11:11:22,2012-10-11 11:11:00,2012-10-11 11:00:00,2012-10-11 11:10:00


In [4]:
array = df_clusters.groupby(["Cluster"]).count().index.tolist()

In [5]:
def getLabel(list_event):
    try:
        temp = list_event.value_counts()
        return temp.keys()[0]
    except:
        return -1

def findPurity(temp_df):
    try:
        label = getLabel(temp_df['Event'])
        n1 = temp_df.loc[temp_df['Event']==label].shape[0]
        n2 = temp_df.shape[0]
        return n1/n2
    except:
        return -1

def getDataframePurity(df,list_predict):
    df_purity = pd.DataFrame()
    df_purity['Cluster'] = np.array([cluster for cluster in list_predict])
#     print(df_purity['Cluster'])
    df_purity['Length_Cluster'] = [df.loc[df.Cluster==temp].shape[0] for temp in df_purity['Cluster']]
    df_purity['Predicted_Event'] = np.array([int(getLabel(df.loc[df['Cluster']==cluster]['Event'])) for cluster in list_predict])
    df_purity['Purity'] = np.array([round(findPurity(df.loc[df['Cluster']==cluster]),1) for cluster in list_predict])
    return df_purity

In [6]:
df_purity = getDataframePurity(df_clusters,array)

In [7]:
df_purity.head()

,Cluster,Length_Cluster,Predicted_Event,Purity
0,0,77,394,1.0
1,1,29,394,1.0
2,2,18,394,1.0
3,3,10,501,1.0
4,4,10,501,1.0


In [8]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
stopword = nltk.corpus.stopwords.words('english')
# Stemming is the process of breaking a word down into its root.
stemmer = SnowballStemmer('english')
stops = set(stopwords.words("english"))
import data_preprocessing_methods as dpm

In [9]:
import collections

def findTopFeatures(temp_df, attribute):
    list_terms = []
    for i in range(temp_df.shape[0]):
        temp_terms = temp_df.iloc[i][attribute]
        for current_terms in temp_terms:
            list_terms.append(current_terms)
    
    counter = collections.Counter(list_terms)
    crucial_words = counter.most_common(3)
    final_list = []
    for i in range(len(crucial_words)):
                final_list.append(crucial_words[i][0])
    return final_list

In [10]:
def createTitle(list_cluster):
    list_rows = []
    for temp_cluster in list_cluster:
        temp_df = df_clusters.loc[df_clusters.Cluster==temp_cluster]
        temp = np.vectorize(dpm.remove_patterns)(temp_df['tweets'],"@[\w]*","#[\w]*","RT")
        temp_df["text"] = np.array([text["input_text"] for text in temp])
        temp_df["hashtag"] = [dpm.list_lower(text["hashtags"]) for text in temp]
        temp_df["user_mentions"] = [text["user_mentions"] for text in temp] 
        temp_df["RT"] = [text["is_RT"] for text in temp]
        temp_df["URL"] = [text["is_URL"] for text in temp]
        
        top_hashtags = findTopFeatures(temp_df,'hashtag')
        top_user_mentions = findTopFeatures(temp_df,'user_mentions')
        top_URLs = findTopFeatures(temp_df,'URL')
        
        temp_df['Tweet_punct'] = temp_df['text'].apply(lambda x: dpm.remove_punct(x))
        temp_df['Tweet_tokenized'] = temp_df['Tweet_punct'].apply(lambda x: dpm.tokenization(x.lower()))
        temp_df['Tweet_nonstop'] = temp_df['Tweet_tokenized'].apply(lambda x: dpm.remove_stopwords(x,stopword))
        temp_df["Stem_sent"] = [dpm.stemming_to_sent(temp,stemmer) for temp in temp_df["Tweet_nonstop"]]
        temp_df["Stem_words"] = [dpm.stemming_to_words(temp,stemmer) for temp in temp_df["Tweet_nonstop"]]
        
        temp_df['urls'] = 'None'
        list_index = temp_df.index.tolist()
        temp_df["clean_data"] = [dpm.new_clean_data(i,temp_df) for i in list_index]
        
#         trainContent = temp_df.clean_data
        trainContent = temp_df.Stem_sent
        from sklearn.feature_extraction.text import CountVectorizer

        headline_vectorizer = CountVectorizer(binary=True,min_df=0.2,ngram_range=(1,1))
        X = headline_vectorizer.fit_transform(trainContent)
        
        max_vector = 0
        for i in range(X.shape[0]):
            count_words = X[i].count_nonzero()
            shape = len(temp_df.Tweet_nonstop.iloc[i])
#             shape = len(temp_df.clean_data.iloc[i])
            if(shape!=0):
                ratio = count_words/shape
                if (ratio>max_vector):
                    max_vector = ratio
                    max_index = i

        title = temp_df.tweets.iloc[max_index]
        cluster_length = temp_df.shape[0]
        timestamp = temp_df.loc[temp_df['Cluster']==temp_cluster].iloc[0]['Timestamp']
#         event = temp_df.loc[temp_df['Cluster']==temp_cluster].iloc[0]['Event']
        event = df_purity.loc[df_purity.Cluster==temp_cluster].iloc[0]['Predicted_Event']
        purity = df_purity.loc[df_purity.Cluster==temp_cluster].iloc[0]['Purity']
        topic = headline_vectorizer.get_feature_names()
        
        temp_row = [title,timestamp,temp_cluster,cluster_length,int(event),purity,topic,
                    top_hashtags,top_user_mentions,top_URLs]
#         print(temp_row)
        list_rows.append(temp_row)
    return list_rows

In [11]:
temp_list = createTitle(array)

In [12]:
summarize_clusters =pd.DataFrame(temp_list,columns=['Title','Timestamp','Cluster','Length','Event','Purity','Topic','Top_Hashtags','Top_Mentions','top_URLs'])

In [13]:
summarize_clusters['Date'] = np.array([list_date_minutes[temp+9] for temp in summarize_clusters['Timestamp']])

In [16]:
summarize_clusters.head(20)

,Title,Timestamp,Cluster,Length,Event,Purity,Topic,Top_Hashtags,Top_Mentions,top_URLs,Date
0,Bet hiphop awards,6,0,77,394,1.0,"[award, bet, hiphop]","[#workhard, #wiz, #bet]","[@sincerelyKEKE89, @50cent, @PrynceShay]",[],2012-10-10 00:15:00
1,2012 HIP~HOP Awards ([BET]) is where it's at....,12,1,29,394,1.0,"[award, bet, hiphop]",[],"[@_TooDopeToCare, @KidFury]",[],2012-10-10 00:21:00
2,Yeen gotta rock wit it but yo girl understand .,14,2,18,394,1.0,"[aint, girl, gotta, rock, understand, wit, ya,...",[#alright],[],[],2012-10-10 00:23:00
3,MEEK MILLZ #HipHopAwards,12,3,10,501,1.0,"[health, kill, meek, millz, nigga, wealth]","[#hiphopawards, #phillyrepresenta, #preach]",[],[],2012-10-10 00:21:00
4,Meek mill got lot chains,15,4,10,501,1.0,"[ahh, bout, chain, dey, flow, got, live, lol, ...",[],[],[],2012-10-10 00:24:00
5,Soo The HipHop Awards Is On ?,16,5,9,394,0.9,"[award, fire, flow, hiphop, right, soo, tune]",[],[],[],2012-10-10 00:25:00
6,Bet awards that's wassup,16,6,10,394,1.0,"[aunti, award, bet, bitchen, catch, grandma, s...",[],[],[],2012-10-10 00:25:00
7,Lyricist of the year : Kendrick Lamar !,17,7,36,19,1.0,"[kendrick, lamar, lyricist, year]","[#blackhippy, #reallyhoe]",[@kendricklamar],[],2012-10-10 00:26:00
8,Meek Mill fine too,19,8,24,501,1.0,"[ass, fine, meek, mill, millz]","[#hiphopawards, #hiphppawards]",[],[],2012-10-10 00:28:00
9,Hiphop awards #bet,20,9,15,394,0.9,"[award, bet, damn, forgot, hiphop]",[#bet],[@iamCuDii],[],2012-10-10 00:29:00
